In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
import streamlit as st
from gensim.models import FastText
import joblib

# Charger le modèle ML , remplacer modele_prix_voiture.pkl par le nom de modele
regression_model = joblib.load("modele_prix_voiture.pkl")  # Assure-toi que ce fichier est dans le même dossier
df_encoded_ref = pd.read_csv("C:/Users/HP G3/Bureau/voitures_preprocessed.csv", encoding='latin1')
marque_modele_map = dict(zip(df_encoded_ref['Marque_Modele'], df_encoded_ref['Marque_Modele_Encoded']))

# Ajouter du CSS personnalisé
st.markdown("""
    <style>
        .title {
            font-size: 30px;
            font-weight: bold;
            color: #3e6b6b;
        }
        .subheader {
            font-size: 18px;
            font-weight: bold;
            color: #3e6b6b;
        }
        .stButton>button {
            background-color: #4CAF50;
            color: white;
            font-size: 16px;
            border-radius: 5px;
            padding: 10px 20px;
        }
        .stButton>button:hover {
            background-color: #45a049;
        }
    </style>
""", unsafe_allow_html=True)

# Fonction pour appliquer les transformations aux données utilisateur
def transform_data(input_data):
    # Appliquer l'encodage ordinale pour la variable "Première main"
    condition_ctg = ['Oui', 'Non']
    condition_encoder = OrdinalEncoder(categories=[condition_ctg])
    input_data['Première main'] = condition_encoder.fit_transform(input_data[['Première main']])

    # Appliquer le One-hot encoding pour les variables non-ordinales
    df_encoded = pd.get_dummies(input_data[['Carburant', 'BoiteàV', 'Origine']], dtype=int)
    
    # Supprimer les colonnes qui vont être encodées
    input_data = input_data.drop(['Carburant', 'BoiteàV', 'Origine'], axis=1)
    
    # Concaténer input_data avec df_encoded
    input_data = pd.concat([input_data, df_encoded], axis=1)
    
    # Supprimer les espaces d'après le nom des variables
    input_data.columns = input_data.columns.str.replace(" ", "_")
    
    # Remplacer l'intervalle du kilométrage par la valeur moyenne
    def get_avrg(val):
        val = str(val)
        if '-' in val:
            bounds = val.split('-')
            return (int(bounds[0]) + int(bounds[1])) / 2
        elif 'Moins' in val:
            number = int(val.split('e')[1])
            return number / 2
        else:
            number = int(val.split('e')[1])
            return number / 2 + 10000

    input_data['Kilométrage'] = input_data['Kilométrage'].apply(get_avrg)
    
    # Combiner la Marque et le modèle
    input_data['Marque_Modele'] = (input_data['Marque'].str.strip() + '_' + input_data['Modèle'].str.strip()).str.replace(' ', '_')
    input_data = input_data.drop(['Marque', 'Modèle'], axis=1)
    input_data['Marque_Modele_Encoded'] = input_data['Marque_Modele'].map(marque_modele_map)
    
    # Si la combinaison n'existe pas, on remplace par la moyenne
    mean_encoding = df_encoded_ref['Marque_Modele_Encoded'].mean()
    input_data['Marque_Modele_Encoded'].fillna(mean_encoding, inplace=True)

    # Gérer les lignes sans équipements
    input_data['équipements'].fillna('', inplace=True)

    # Gérer les caractères dans équipements
    input_data['équipements_clean'] = input_data['équipements'].str.lower().str.replace('/', ' ').str.replace(',', '').str.split()

    # Entraîner un modèle FastText directement sur les données
    sentences = input_data['équipements_clean'].tolist()
    fasttext_model = FastText(sentences, vector_size=100, window=3, min_count=1, workers=4)

    # Fonction pour vectoriser les équipements
    def vectorize_equipements(equipements_list, model):
        vectors = [model.wv[word] for word in equipements_list if word in model.wv]
        if vectors:
            return np.mean(vectors, axis=0)
        else:
            return np.zeros(model.vector_size)

    # Appliquer le word embedding utilisant FastText
    input_data['équipements_vector'] = input_data['équipements_clean'].apply(lambda x: vectorize_equipements(x, model=fasttext_model))
    input_data = input_data.drop(['équipements', 'équipements_clean'], axis=1)
    
    # Retourner les données transformées
    return input_data
    
# Titre de l'application
st.markdown('<p class="title">Estimation du Prix d\'un Véhicule</p>', unsafe_allow_html=True)

# Champs pour le kilométrage
st.subheader("Kilométrage")

# Récupérer les données de l'utilisateur via Streamlit
Kilometrage_min = st.number_input("Kilométrage minimum (en km)", min_value=0, step=1000)
Kilometrage_max = st.number_input("Kilométrage maximum (en km)", min_value=Kilometrage_min, step=1000)
Nombre_de_portes = st.selectbox("Nombre de portes", [2, 3, 4, 5, 6])
Premiere_main = st.radio("Première main", ["Oui", "Non"])
Puissance_fiscale = st.number_input("Puissance fiscale (en chevaux)", min_value=1, step=1)
Année_de_fabriquation = st.number_input("Année du modèle", min_value=1990, max_value=2025, step=1)
Carburant = st.selectbox("Carburant", ["Diesel", "Electrique", "Essence", "Hybride", "LPG"])
Boite_vitesse = st.radio("Boîte à vitesse", ["Automatique", "Manuelle"])
Origine = st.selectbox("Origine", ["Importée neuve", "Pas encore dédouanée", "WW au Maroc", "Dédouanée"])
Marque = st.text_input("Marque du véhicule")
Modèle = st.text_input("Modèle du véhicule")
Equipements = st.text_area("Équipements (séparez les équipements par des virgules)")

# Créer un DataFrame à partir des données utilisateur
input_data = pd.DataFrame({
    'Kilométrage': [f"{Kilometrage_min}-{Kilometrage_max}"],
    'Nombre_de_portes': [Nombre_de_portes],
    'Première main': [Premiere_main],
    'Puissance_fiscale': [Puissance_fiscale],
    'Année': [Année_de_fabriquation],
    'Carburant': [Carburant],
    'BoiteàV': [Boite_vitesse],
    'Origine': [Origine],
    'Marque': [Marque],
    'Modèle': [Modèle],
    'équipements': [Equipements]
})

if st.button('Prédire le prix'):
    # Appliquer les transformations
    transformed_data = transform_data(input_data)

    # Supprimer les colonnes inutiles (ex. : Marque_Modele si pas dans les features du modèle)
    if 'Marque_Modele' in transformed_data.columns:
        transformed_data = transformed_data.drop(['Marque_Modele'], axis=1)

    # Prédire
    predicted_price = regression_model.predict(transformed_data)

    # Afficher le prix estimé
    st.subheader(f"Prix estimé : {predicted_price[0]:,.2f} MAD")
